In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.cross_decomposition import PLSRegression

In [16]:
housPrice_df = pd.read_csv('MELBOURNE_HOUSE_PRICES_LESS.csv')
housPrice_df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Postcode,Regionname,Propertycount,Distance,CouncilArea
0,Abbotsford,49 Lithgow St,3,h,1490000.0,S,Jellis,1/04/2017,3067,Northern Metropolitan,4019,3.0,Yarra City Council
1,Abbotsford,59A Turner St,3,h,1220000.0,S,Marshall,1/04/2017,3067,Northern Metropolitan,4019,3.0,Yarra City Council
2,Abbotsford,119B Yarra St,3,h,1420000.0,S,Nelson,1/04/2017,3067,Northern Metropolitan,4019,3.0,Yarra City Council
3,Aberfeldie,68 Vida St,3,h,1515000.0,S,Barry,1/04/2017,3040,Western Metropolitan,1543,7.5,Moonee Valley City Council
4,Airport West,92 Clydesdale Rd,2,h,670000.0,S,Nelson,1/04/2017,3042,Western Metropolitan,3464,10.4,Moonee Valley City Council


In [17]:
housPrice_df = housPrice_df.drop(['Address', 'SellerG', 'Date', 'CouncilArea'], 1)
lables = housPrice_df.columns[:]
lables

Index(['Suburb', 'Rooms', 'Type', 'Price', 'Method', 'Postcode', 'Regionname',
       'Propertycount', 'Distance'],
      dtype='object')

In [25]:
Method = pd.get_dummies(housPrice_df['Method'])
Method.head()

,PI,PN,S,SA,SN,SP,SS,VB,W
0,0,0,1,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0


In [27]:
houseType = pd.get_dummies(housPrice_df['Type'])
houseType.head()

,h,t,u
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [22]:
Suburb = pd.get_dummies(housPrice_df['Suburb'])
Suburb.head()

,Abbotsford,Aberfeldie,Airport West,Albanvale,Albert Park,Albion,Alphington,Altona,Altona Meadows,Altona North,...,Woori Yallock,Wyndham Vale,Yallambie,Yan Yean,Yarra Glen,Yarra Junction,Yarrambat,Yarraville,croydon,viewbank
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
Regionname = pd.get_dummies(housPrice_df['Regionname'])
Regionname.head()

,Eastern Metropolitan,Eastern Victoria,Northern Metropolitan,Northern Victoria,South-Eastern Metropolitan,Southern Metropolitan,Western Metropolitan,Western Victoria
0,0,0,1,0,0,0,0,0
1,0,0,1,0,0,0,0,0
2,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,1,0


In [48]:
housPrice_df['Price'] = np.log(housPrice_df['Price'])
housPrice_df = pd.concat([housPrice_df, Suburb, Method, houseType, Regionname], axis=1).drop(['Type', 'Method', 'Suburb', 'Regionname'], 1).dropna()
housPrice_df = housPrice_df.drop([53116, 57390])
housPrice_df.head()

,Rooms,Price,Postcode,Propertycount,Distance,Abbotsford,Aberfeldie,Airport West,Albanvale,Albert Park,...,t,u,Eastern Metropolitan,Eastern Victoria,Northern Metropolitan,Northern Victoria,South-Eastern Metropolitan,Southern Metropolitan,Western Metropolitan,Western Victoria
0,3.0,2.654248,3067.0,4019.0,3.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,3.0,2.640083,3067.0,4019.0,3.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3.0,2.650857,3067.0,4019.0,3.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,3.0,2.655417,3040.0,1543.0,7.5,0.0,1.0,0.0,0.0,0.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2.0,2.596376,3042.0,3464.0,10.4,0.0,0.0,1.0,0.0,0.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [51]:
X = housPrice_df.drop('Price', 1)
y = housPrice_df['Price']

In [54]:
# Instantiate and fit our model.
regr = linear_model.LinearRegression()

regr.fit(X, y)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, y))


Coefficients: 
 [  1.50576556e-02   4.33584869e+02  -5.22292823e+00 ...,   2.01294997e+05
   5.40952310e+05   5.45215360e+04]

Intercept: 
 -11126976.4313

R-squared:
0.761985924743


In [55]:
cross_val_score(regr, X, y, cv=10)

array([ -7.52448818e+11,  -1.44594183e+11,  -7.88403927e+08,
        -8.74473903e+08,  -5.84702735e+10,  -1.18689282e+10,
        -2.39811626e+11,  -3.37616691e+12,  -2.54073216e+11,
        -4.21152931e+10])

# Partial Least Squares Regression

In [56]:
# Fit a linear model using Partial Least Squares Regression.
# Reduce feature space to 9 dimensions.
pls1 = PLSRegression(n_components=9)

# Reduce X to R(X) and regress on y.
pls1.fit(X, y)

# Save predicted values.
Y_PLS_pred = pls1.predict(X)
print('R-squared PLSR:', pls1.score(X, y))

R-squared PLSR: 0.761985895603


In [57]:
cross_val_score(pls1, X, y, cv=5)

array([ 0.75472749,  0.76224066,  0.75784627,  0.75980106,  0.75154614])